# 4 Dataset Classification

In [3]:
#Importing the libraries to watch the 'fits' image and get the data array
import astropy
#import plotly.graph_objects as go
from astropy.io import fits
#Importing a library that is useful to read the original file
import pandas as pd
import pylab as plb
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
#Importing a visual library with some illustrative set up
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors 
from matplotlib import cm
import numpy as np
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.decomposition import PCA
import math 
import seaborn as sns
from sklearn.linear_model import LogisticRegression
plt.style.use('fivethirtyeight')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
#plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (16, 8) 
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
plt.rcParams["axes.grid"] = False



In [4]:
data=pd.read_csv('star.txt',sep='\s+')

In [5]:
x=np.array(data.F606W-data.F814W)
y=np.array(data.F606W)

In [6]:
data.head()

,#ID,X,Y,F606W,error,F814W,error.1,Chi,Sharp
0,8,4462.947,140.859,28.197,0.1036,27.127,0.1068,6.106,0.7310
1,120,5002.486,186.138,20.843,0.0552,19.815,0.0661,10.690,0.4000
2,165,5003.574,195.421,27.014,0.2194,26.068,0.1349,10.651,0.1940
3,226,4448.493,207.417,30.193,0.3641,28.270,0.1636,4.059,-1.9585
4,351,5040.717,224.366,28.935,0.1669,28.941,0.2464,4.274,-0.7720


In [7]:
notar=data.drop(columns=['Sharp','#ID'])

# P.C.A. Excluding Space

In [6]:
notar=data.drop(columns=['Sharp','#ID','X','Y'])

In [9]:
notar.head()

,F606W,error,F814W,error.1,Chi
0,28.197,0.1036,27.127,0.1068,6.106
1,20.843,0.0552,19.815,0.0661,10.690
2,27.014,0.2194,26.068,0.1349,10.651
3,30.193,0.3641,28.270,0.1636,4.059
4,28.935,0.1669,28.941,0.2464,4.274


In [7]:
pca=PCA(n_components=3)
pca=pca.fit(notar)
pca_data=pd.DataFrame(pca.transform(notar))

In [8]:
pca_data=pca_data.rename(columns={0:'FirstComponent',1:'SecondComponent',2:'ThirdComponent'})

In [57]:
pca_data['X']=data.X
pca_data['Y']=data.Y

In [58]:
pca_data['Sharp']=data.Sharp

In [59]:
pca_data.head()

,FirstComponent,SecondComponent,ThirdComponent,X,Y,Sharp
0,-3.348246,3.558359,0.052565,4462.947,140.859,0.7310
1,6.885136,8.439184,0.189286,5002.486,186.138,0.4000
2,-1.896787,8.148357,0.145220,5003.574,195.421,0.1940
3,-5.504066,1.449171,-0.564914,4448.493,207.417,-1.9585
4,-5.110070,1.678896,0.804773,5040.717,224.366,-0.7720


In [60]:
#pca_data=pca_data.drop(columns=['Sharp'])

# 2-class: Positive or Negative

# 1. PCA Linear Classifier 

In [9]:
data['SharpSign']=data.Sharp.apply(np.sign)
data['SharpSign']=data['SharpSign'].replace(0,1)

In [10]:
pca_data['Target']=data['SharpSign']

In [303]:
pca_data.head()

,FirstComponent,SecondComponent,ThirdComponent,X,Y,Sharp,Target
0,-3.348246,3.558359,0.052565,4462.947,140.859,0.7310,1.0
1,6.885136,8.439184,0.189286,5002.486,186.138,0.4000,1.0
2,-1.896787,8.148357,0.145220,5003.574,195.421,0.1940,1.0
3,-5.504066,1.449171,-0.564914,4448.493,207.417,-1.9585,-1.0
4,-5.110070,1.678896,0.804773,5040.717,224.366,-0.7720,-1.0


In [11]:
from sklearn.svm import LinearSVC as SVC
from sklearn.model_selection import train_test_split


In [325]:
X=pca_data.drop(columns=['Sharp','Target'])

In [326]:
y=pca_data.Target

In [327]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [328]:
X_traint,X_train_val,y_traint,y_train_val= train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
C_SCORE=[]
c=np.arange(0.2,1.2,0.2)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

In [346]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 0.2 and 1.0

the best score has been obtained with 0.6000000000000001
and it is 0.6128444822143985


In [347]:
C_SCORE=[]
c=np.arange(1,11,1)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

1 coefficient has been adopted
2 coefficient has been adopted
3 coefficient has been adopted
4 coefficient has been adopted
5 coefficient has been adopted
6 coefficient has been adopted
7 coefficient has been adopted
8 coefficient has been adopted
9 coefficient has been adopted
10 coefficient has been adopted


In [349]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 1 and 10

the best score has been obtained with 7
and it is 0.6253490348427826


In [343]:
C_SCORE=[]
c=np.arange(10,110,10)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

10 coefficient has been adopted
20 coefficient has been adopted
30 coefficient has been adopted
40 coefficient has been adopted
50 coefficient has been adopted
60 coefficient has been adopted
70 coefficient has been adopted
80 coefficient has been adopted
90 coefficient has been adopted
100 coefficient has been adopted


In [344]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 10 and 100

the best score has been obtained with 30
and it is 0.6247420177248998


In [386]:
OPT_C=[0.6,7,30]

In [388]:
FIN_SCORE=[]
for opt_C in OPT_C:
    clf=SVC(C=opt_c)
    clf.fit(X_train,y_train)
    fin_score=clf.score(X_test,y_test)
    FIN_SCORE.append(fin_score)

In [391]:
FIN_SCORE=np.array(FIN_SCORE)

In [392]:
fin_score=FIN_SCORE.max()

In [394]:
print ('The PCA dataset gave a best classification with ' + str(fin_score*100)+ '% of accuracy with a linear classifier')

The PCA dataset gave a best classification with 62.849650349650354% of accuracy with a linear classifier


# 1.2 Dataset Linear Classifier

In [413]:
X=data.drop(columns=['SharpSign'])

In [414]:
X=pca_data.drop(columns=['Sharp','Target'])

In [415]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [416]:
X_traint,X_train_val,y_traint,y_train_val= train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [417]:
C_SCORE=[]
c=np.arange(0.2,1.2,0.2)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

0.2 coefficient has been adopted
0.4 coefficient has been adopted
0.6000000000000001 coefficient has been adopted
0.8 coefficient has been adopted
1.0 coefficient has been adopted


In [418]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 0.2 and 1.0

the best score has been obtained with 1.0
and it is 0.6195216705111084


In [419]:
C_SCORE=[]
c=np.arange(1,11,1)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

1 coefficient has been adopted
2 coefficient has been adopted
3 coefficient has been adopted
4 coefficient has been adopted
5 coefficient has been adopted
6 coefficient has been adopted
7 coefficient has been adopted
8 coefficient has been adopted
9 coefficient has been adopted
10 coefficient has been adopted


In [420]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 1 and 10

the best score has been obtained with 9
and it is 0.6219497389826393


In [421]:
C_SCORE=[]
c=np.arange(10,110,10)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

10 coefficient has been adopted
20 coefficient has been adopted
30 coefficient has been adopted
40 coefficient has been adopted
50 coefficient has been adopted
60 coefficient has been adopted
70 coefficient has been adopted
80 coefficient has been adopted
90 coefficient has been adopted
100 coefficient has been adopted


In [422]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 10 and 100

the best score has been obtained with 70
and it is 0.6226781595240986


In [423]:
OPT_C=[1,70]

In [424]:
FIN_SCORE=[]
for opt_C in OPT_C:
    clf=SVC(C=opt_c)
    clf.fit(X_train,y_train)
    fin_score=clf.score(X_test,y_test)
    FIN_SCORE.append(fin_score)

In [425]:
FIN_SCORE=np.array(FIN_SCORE)

In [426]:
fin_score=FIN_SCORE.max()

In [427]:
print ('The PCA dataset gave a best classification with ' + str(fin_score*100)+ '% of accuracy with a linear classifier')

The PCA dataset gave a best classification with 61.34421134421134% of accuracy with a linear classifier


# 1.3 Best Features Classification Original Data

In [433]:
opt_data=data.drop(columns=['#ID','X','Y','Chi','Sharp','F606W'])

In [434]:
opt_data.head()

,error,F814W,error.1,SharpSign
0,0.1036,27.127,0.1068,1.0
1,0.0552,19.815,0.0661,1.0
2,0.2194,26.068,0.1349,1.0
3,0.3641,28.270,0.1636,-1.0
4,0.1669,28.941,0.2464,-1.0


In [436]:
X=opt_data.drop(columns=['SharpSign'])

In [437]:
data['SharpSign']=data.Sharp.apply(np.sign)
data['SharpSign']=data['SharpSign'].replace(0,1)
y=data.SharpSign

In [438]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [439]:
X_traint,X_train_val,y_traint,y_train_val= train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [440]:
C_SCORE=[]
c=np.arange(0.2,1.2,0.2)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

0.2 coefficient has been adopted
0.4 coefficient has been adopted
0.6000000000000001 coefficient has been adopted
0.8 coefficient has been adopted
1.0 coefficient has been adopted


In [441]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 0.2 and 1.0

the best score has been obtained with 0.2
and it is 0.6895714459147748


In [442]:
C_SCORE=[]
c=np.arange(1,11,1)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

1 coefficient has been adopted
2 coefficient has been adopted
3 coefficient has been adopted
4 coefficient has been adopted
5 coefficient has been adopted
6 coefficient has been adopted
7 coefficient has been adopted
8 coefficient has been adopted
9 coefficient has been adopted
10 coefficient has been adopted


In [443]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 1 and 10

the best score has been obtained with 1
and it is 0.6692970741774918


In [444]:
C_SCORE=[]
c=np.arange(10,110,10)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

10 coefficient has been adopted
20 coefficient has been adopted
30 coefficient has been adopted
40 coefficient has been adopted
50 coefficient has been adopted
60 coefficient has been adopted
70 coefficient has been adopted
80 coefficient has been adopted
90 coefficient has been adopted
100 coefficient has been adopted


In [445]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 10 and 100

the best score has been obtained with 40
and it is 0.6248634211484764


In [447]:
OPT_C=[0.2,1,40]

In [448]:
FIN_SCORE=[]
for opt_C in OPT_C:
    clf=SVC(C=opt_c)
    clf.fit(X_train,y_train)
    fin_score=clf.score(X_test,y_test)
    FIN_SCORE.append(fin_score)

In [449]:
FIN_SCORE=np.array(FIN_SCORE)

In [450]:
fin_score=FIN_SCORE.max()

In [451]:
print ('The PCA dataset gave a best classification with ' + str(fin_score*100)+ '% of accuracy with a linear classifier')

The PCA dataset gave a best classification with 62.771950271950274% of accuracy with a linear classifier


# 1.4 Best Feature Classification PCA data

In [452]:
opt_data=pca_data[['FirstComponent','SecondComponent','ThirdComponent']]

In [453]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [454]:
X_traint,X_train_val,y_traint,y_train_val= train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [455]:
C_SCORE=[]
c=np.arange(0.2,1.2,0.2)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

0.2 coefficient has been adopted
0.4 coefficient has been adopted
0.6000000000000001 coefficient has been adopted
0.8 coefficient has been adopted
1.0 coefficient has been adopted


In [456]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 0.2 and 1.0

the best score has been obtained with 0.2
and it is 0.6904212698798106


In [457]:
C_SCORE=[]
c=np.arange(1,11,1)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

1 coefficient has been adopted
2 coefficient has been adopted
3 coefficient has been adopted
4 coefficient has been adopted
5 coefficient has been adopted
6 coefficient has been adopted
7 coefficient has been adopted
8 coefficient has been adopted
9 coefficient has been adopted
10 coefficient has been adopted


In [458]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 1 and 10

the best score has been obtained with 1
and it is 0.6713609323782931


In [459]:
C_SCORE=[]
c=np.arange(10,110,10)
for c_value in c:
    clf=SVC(C=c_value)
    clf.fit(X_traint,y_traint)
    sc=clf.score(X_train_val,y_train_val)
    C_SCORE.append(sc)
    print(str(c_value) + ' coefficient has been adopted' )

10 coefficient has been adopted
20 coefficient has been adopted
30 coefficient has been adopted
40 coefficient has been adopted
50 coefficient has been adopted
60 coefficient has been adopted
70 coefficient has been adopted
80 coefficient has been adopted
90 coefficient has been adopted
100 coefficient has been adopted


In [460]:
MIN_VAL=np.array(C_SCORE).max()
MIN_C=c[np.array(C_SCORE).argmax()]
print ('In the range ' + str(c.min())+ ' and ' +str(c.max())+ '\n')
print ('the best score has been obtained with ' + str(MIN_C))
print ('and it is ' + str(MIN_VAL))

In the range 10 and 100

the best score has been obtained with 70
and it is 0.6231637732184048


In [461]:
OPT_C=[0.2,1,70]

In [462]:
FIN_SCORE=[]
for opt_C in OPT_C:
    clf=SVC(C=opt_c)
    clf.fit(X_train,y_train)
    fin_score=clf.score(X_test,y_test)
    FIN_SCORE.append(fin_score)

In [463]:
FIN_SCORE=np.array(FIN_SCORE)

In [464]:
fin_score=FIN_SCORE.max()

In [465]:
print ('The PCA dataset gave a best classification with ' + str(fin_score*100)+ '% of accuracy with a linear classifier')

The PCA dataset gave a best classification with 63.238150738150736% of accuracy with a linear classifier
